In [1]:
%load_ext autoreload 
%autoreload 2

# imports and setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os 

# import and instanciate config file to get access to paths
os.chdir(os.path.dirname(os.getcwd()))
from config.config import Config
config = Config()

# import utils functions
from src.utils.process_books import process_books
from src.pipelines.preprocessor import Preprocessor

In [2]:
from pandas import read_csv


df_init = read_csv(config.DATA_PROCESSED_PATH + 'books_processed.csv')
df_init

,Author,Book
0,Twain Mark,﻿The Project Gutenberg EBook of Chapters from ...
1,Plato,"﻿The Project Gutenberg EBook of Sophist, by Pl..."
2,Twain Mark,﻿The Project Gutenberg EBook of On the Decay o...
3,Dante Alighieri,"﻿The Project Gutenberg eBook of Pericles, by W..."
4,Shelley Mary Wollstonecraft,﻿The Project Gutenberg eBook of As You Like It...
...,...,...
431,Various,﻿The Project Gutenberg EBook of Life On The Mi...
432,Twain Mark,﻿The Project Gutenberg EBook of A Tramp Abroad...
433,Austen Jane,﻿The Project Gutenberg eBook of Sense and Sens...
434,Luther Martin,﻿\nProject Gutenberg Etext of The Rape of Lucr...


In [3]:
from src.pipelines.preprocessor import Preprocessor

preprocessor = Preprocessor()

In [4]:
df = preprocessor.body_preprocessor(df_init)
df

,Author,Book
0,Twain Mark,"Produced by Betsie Bush, Chuck Greif, Martin P..."
1,Plato,Produced by Sue Asscher\n\n\n\n\n\nSOPHIST\n\n...
2,Twain Mark,ON THE DECAY OF THE ART OF LYING\n\nby Mark Tw...
3,Dante Alighieri,"cover \n\n\n\nPERICLES, PRINCE OF TYRE\n\n\n\n..."
4,Shelley Mary Wollstonecraft,cover \n\n\n\n\nAS YOU LIKE IT\n\nby William S...
...,...,...
431,Various,Produced by David Widger\n\n\n\n\n\n ...
432,Twain Mark,"Produced by Anonymous Volunteers, John Greenma..."
433,Austen Jane,Transcriber's Note:\n\nThe Table of Contents i...
434,Luther Martin,﻿\nProject Gutenberg Etext of The Rape of Lucr...


In [5]:
df = preprocessor.clean_body_preprocessor(df)
df

,Author,Book
0,Twain Mark,"produced by betsie bush, chuck greif, martin p..."
1,Plato,produced by sue asscher\n\n\n\n\n\nsophist\n\n...
2,Twain Mark,on the decay of the art of lying\n\nby mark tw...
3,Dante Alighieri,"cover \n\n\n\npericles, prince of tyre\n\n\n\n..."
4,Shelley Mary Wollstonecraft,cover \n\n\n\n\nas you like it\n\nby william s...
...,...,...
431,Various,produced by david widger\n\n\n\n\n\n ...
432,Twain Mark,"produced by anonymous volunteers, john greenma..."
433,Austen Jane,transcriber's note:\n\nthe table of contents i...
434,Luther Martin,﻿\nproject gutenberg etext of the rape of lucr...


In [6]:
df = preprocessor.chunk_text_preprocessor(df)
df

427 chunks created
176 chunks created
8 chunks created
77 chunks created
90 chunks created
80 chunks created
88 chunks created
55 chunks created
636 chunks created
3757 chunks created
285 chunks created
12 chunks created
116 chunks created
119 chunks created
535 chunks created
39 chunks created
84 chunks created
96 chunks created
275 chunks created
364 chunks created
46 chunks created
91 chunks created
82 chunks created
84 chunks created
57 chunks created
95 chunks created
104 chunks created
87 chunks created
92 chunks created
431 chunks created
844 chunks created
110 chunks created
36 chunks created
115 chunks created
540 chunks created
89 chunks created
32 chunks created
540 chunks created
115 chunks created
86 chunks created
103 chunks created
121 chunks created
32 chunks created
111 chunks created
207 chunks created
617 chunks created
52 chunks created
26 chunks created
75 chunks created
79 chunks created
115 chunks created
107 chunks created
258 chunks created
68 chunks created
66

,Author,Book
0,Twain Mark,"produced by betsie bush, chuck greif, martin p..."
1,Twain Mark,which he sees his own life reflected and set d...
2,Twain Mark,"believed it would, and asked me if i meant to ..."
3,Twain Mark,"tradition, this one or another--geoffrey cleme..."
4,Twain Mark,of clemenses who said they had examined the re...
...,...,...
92892,Burnett Frances Hodgson,"one another, and give rise to a similar opposi..."
92893,Burnett Frances Hodgson,"she desires to create, but only in what will p..."
92894,Burnett Frances Hodgson,"good and their opposites, which is true and co..."
92895,Burnett Frances Hodgson,"medicine which art prescribes for them, and of..."


In [7]:
df = preprocessor.encode_label_preprocessor(df)
df

,Author,Book
0,207,"produced by betsie bush, chuck greif, martin p..."
1,207,which he sees his own life reflected and set d...
2,207,"believed it would, and asked me if i meant to ..."
3,207,"tradition, this one or another--geoffrey cleme..."
4,207,of clemenses who said they had examined the re...
...,...,...
92892,33,"one another, and give rise to a similar opposi..."
92893,33,"she desires to create, but only in what will p..."
92894,33,"good and their opposites, which is true and co..."
92895,33,"medicine which art prescribes for them, and of..."


In [11]:
from src.pipelines.preproc.preprocv1 import build_preprocessor_pipeline

In [15]:
from sklearn import set_config; set_config(display='diagram')
pipe = build_preprocessor_pipeline()
pipe


Pipeline(steps=[ColumnTransformer(transformers=[('text_pipeline',
                                                 Pipeline(steps=[('body',
                                                                  FunctionTransformer(func=<bound method Preprocessor.body_preprocessor of <src.pipelines.preprocessor.Preprocessor object at 0x2a6417910>>)),
                                                                 ('clean_body',
                                                                  FunctionTransformer(func=<bound method Preprocessor.clean_body_preprocessor of <src.pipelines.preprocessor.Preprocessor object at 0x2a6417910>>))])),
                                                ('author_pipeline',
                                                 Pipeline(steps=[('encoder',
                                                                  FunctionTransformer(func=<bound method Preprocessor.encode_label_preprocessor of <src.pipelines.preprocessor.Preprocessor object at 0x2a6417910>>))]))])])

In [14]:
pipe.fit_transform(df_init)

TypeError: cannot unpack non-iterable ColumnTransformer object

In [ ]:
df_init

,Author,Book
0,207,"produced by betsie bush, chuck greif, martin p..."
1,163,produced by sue asscher\n\n\n\n\n\nsophist\n\n...
2,207,on the decay of the art of lying\n\nby mark tw...
3,53,"cover \n\n\n\npericles, prince of tyre\n\n\n\n..."
4,183,cover \n\n\n\n\nas you like it\n\nby william s...
...,...,...
431,210,produced by david widger\n\n\n\n\n\n ...
432,207,"produced by anonymous volunteers, john greenma..."
433,10,transcriber's note:\n\nthe table of contents i...
434,126,﻿\nproject gutenberg etext of the rape of lucr...


In [ ]:
df_init = read_csv(config.DATA_PROCESSED_PATH + 'books_processed.csv')
df = preprocessor.body_preprocessor(df_init)
df = preprocessor.clean_body_preprocessor(df)
df = preprocessor.chunk_text_preprocessor(df)
df = preprocessor.encode_label_preprocessor(df)
df


Index(['Author', 'Book'], dtype='object')
427 chunks created
176 chunks created
8 chunks created
77 chunks created
90 chunks created
80 chunks created
88 chunks created
55 chunks created
636 chunks created
3757 chunks created
285 chunks created
12 chunks created
116 chunks created
119 chunks created
535 chunks created
39 chunks created
84 chunks created
96 chunks created
275 chunks created
364 chunks created
46 chunks created
91 chunks created
82 chunks created
84 chunks created
57 chunks created
95 chunks created
104 chunks created
87 chunks created
92 chunks created
431 chunks created
844 chunks created
110 chunks created
36 chunks created
115 chunks created
540 chunks created
89 chunks created
32 chunks created
540 chunks created
115 chunks created
86 chunks created
103 chunks created
121 chunks created
32 chunks created
111 chunks created
207 chunks created
617 chunks created
52 chunks created
26 chunks created
75 chunks created
79 chunks created
115 chunks created
107 chunks creat

,Author,Book
0,207,"produced by betsie bush, chuck greif, martin p..."
1,207,which he sees his own life reflected and set d...
2,207,"believed it would, and asked me if i meant to ..."
3,207,"tradition, this one or another--geoffrey cleme..."
4,207,of clemenses who said they had examined the re...
...,...,...
92892,33,"one another, and give rise to a similar opposi..."
92893,33,"she desires to create, but only in what will p..."
92894,33,"good and their opposites, which is true and co..."
92895,33,"medicine which art prescribes for them, and of..."


In [ ]:
# save df to csv 
df.to_csv(config.DATA_FINAL_PATH + 'books_preprocessed.csv', index=False)

In [ ]:
# word2vec embedding

from gensim.models import Word2Vec

wordtovec = Word2Vec(df['Book'], window=10, workers=5)

In [ ]:
def remove_da_shit(words):
    return [word for word in words if word not in ['ἅ', '•', 'ÿ', 'ὦ', 'ἠ', '×', '°', 'ß', '᾽', 'š', '\x92', 'ỏ']]

df['Book'] = df['Book'].apply(remove_da_shit)

df['Book'] = df['Book'].apply(lambda x: [wordtovec.wv[word] for word in x if word in wordtovec.wv])
df



,Author,Book
0,207,"[[-0.14814544, 0.17401853, -0.41310522, -0.211..."
1,207,"[[0.6881956, 0.99037874, 0.2590469, 0.265691, ..."
2,207,"[[0.4794665, 0.52229464, 0.0414591, 0.8494552,..."
3,207,"[[0.23006427, 0.5083755, 0.17851238, -0.018575..."
4,207,"[[0.46350753, 0.66330725, 0.61609185, 0.204386..."
...,...,...
92892,33,"[[0.46350753, 0.66330725, 0.61609185, 0.204386..."
92893,33,"[[0.17210265, 0.5945774, 0.16267185, 0.2602231..."
92894,33,"[[-0.038096186, 0.8723225, 0.16824491, 1.13946..."
92895,33,"[[-0.008179311, 1.0293553, 0.37904462, 0.29931..."


In [ ]:
# save df to csv

df.to_csv(config.DATA_FINAL_PATH + 'books_preprocessed_vec.csv', index=False)